In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Load dataset
data = pd.read_csv('processed_emotions_dataset_2.csv', index_col=0)
data = data.dropna()  # Drop missing values
docs = data['text']  # Text column from dataset
labels = data['label']  # Labels (emotions)

# Define stopwords and additional custom stopwords
stop_words = set(stopwords.words('english'))
negation_words = ['not', "don't", 'no', 'never', "can't", "won't"]
additional_stopwords = {
    'really', 'very', 'totally', 'absolutely', 'so', 'quite', 'extremely',
    'incredibly', 'super', 'definitely', 'omg', 'lol', 'lmao', 'brb', 'fml',
    'idk', 'smh', 'tbh', 'like', 'you', 'know', 'um', 'literally', 'actually',
    'basically', 'kind', 'sorta', 'might', 'would', 'could', 'can', 'should',
    'not', 'no', 'don’t', 'never', 'nothing', 'feel', 'felt', 'wish', 'hope',
    'maybe', 'perhaps', 'im', 'uh', 'oh', 'wow', 'ouch', 'eh', 'whoa', 'yikes'
}
stop_words.update(additional_stopwords)

# Preprocessing function for tokenization and filtering
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = []
    negation = False
    for word in tokens:
        if word in negation_words:
            negation = not negation  # Flip negation status
            continue  # Skip the negation word itself
        if word.isalpha() and word not in stop_words:
            if negation:
                word = 'not_' + word  # Mark the word as negated
            filtered_tokens.append(word)
            negation = False  # Reset negation after the word
    return ' '.join(filtered_tokens)

# Preprocess the documents
preprocessed_docs = [preprocess_text(doc) for doc in docs]

# Tokenization and padding setup
max_words = 10000  # Maximum number of words in the vocabulary
max_sequence_length = 100  # Maximum length of each input sequence

# Tokenizer to convert text into sequences of integers
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(preprocessed_docs)
sequences = tokenizer.texts_to_sequences(preprocessed_docs)

# Pad sequences to ensure consistent input size
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to categorical (one-hot encoding)
y = to_categorical(labels, num_classes=6)  # 6 emotion classes

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 6 emotions classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Optional: Make predictions on a new example
test_text = "I am feeling great today!"
processed_test_text = preprocess_text(test_text)
test_sequence = tokenizer.texts_to_sequences([processed_test_text])
test_padded = pad_sequences(test_sequence, maxlen=max_sequence_length)
prediction = model.predict(test_padded)
predicted_label = np.argmax(prediction, axis=1)
print(f"Predicted Label for the test text: {predicted_label[0]}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
10506/10506 ━━━━━━━━━━━━━━━━━━━━ 737s 70ms/step - accuracy: 0.8216 - loss: 0.4676 - val_accuracy: 0.9171 - val_loss: 0.1581
Epoch 2/5
10506/10506 ━━━━━━━━━━━━━━━━━━━━ 743s 70ms/step - accuracy: 0.9161 - loss: 0.1752 - val_accuracy: 0.9175 - val_loss: 0.1542
Epoch 3/5
10506/10506 ━━━━━━━━━━━━━━━━━━━━ 695s 66ms/step - accuracy: 0.9220 - loss: 0.1608 - val_accuracy: 0.9183 - val_loss: 0.1552
Epoch 4/5
10506/10506 ━━━━━━━━━━━━━━━━━━━━ 735s 65ms/step - accuracy: 0.9248 - loss: 0.1540 - val_accuracy: 0.9178 - val_loss: 0.1622
Epoch 5/5
10506/10506 ━━━━━━━━━━━━━━━━━━━━ 689s 66ms/step - accuracy: 0.9264 - loss: 0.1486 - val_accuracy: 0.9195 - val_loss: 0.1641
2627/2627 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9188 - loss: 0.1650
Test accuracy: 91.95%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Predicted Label for the test text: 1
